<a href="https://colab.research.google.com/github/chabryl/AI-Redirect-Mapping/blob/main/generate_rewrite_rules.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
from urllib.parse import urlparse
from datetime import datetime
from google.colab import files

# Schritt 1: CSV-Datei hochladen
uploaded = files.upload()

# CSV-Datei laden
df = pd.read_csv(list(uploaded.keys())[0])

# Erste Zeilen anzeigen
print(df.head())

# Schritt 2: Konvertiere die Spalte 'similarity_score' zu float
df['similarity_score'] = pd.to_numeric(df['similarity_score'], errors='coerce')

# Definieren des Schwellenwerts für den Similarity Score
score_threshold = 0.91

# URLs filtern, die sich für redirects.txt qualifizieren
df_redirects = df[
    (df['origin_language'] == df['matched_language']) &
    (df['similarity_score'] >= score_threshold) &
    (df['origin_url'] != df['matched_url'])  # Kein Self-Redirect
]

# Schritt 3: Funktion zur Umwandlung absoluter URLs in relative URLs
def convert_to_relative(url):
    """Konvertiert eine absolute URL zu einer relativen URL."""
    parsed_url = urlparse(url)
    return parsed_url.path

# Generieren der Rewrite-Regeln
def generate_rewrite_rule(origin_url, matched_url):
    relative_origin = convert_to_relative(origin_url)
    relative_matched = convert_to_relative(matched_url)

    # Vermeiden von Self-Redirects und Schleifen
    if relative_origin != relative_matched:
        return f"rewrite ^{relative_origin}$ {relative_matched} redirect;"
    return None

# Erstellen der Rewrite-Regeln für alle qualifizierten URLs
rewrite_rules = df_redirects[['origin_url', 'matched_url']].apply(
    lambda row: generate_rewrite_rule(row['origin_url'], row['matched_url']), axis=1)

# Bereinigen der Regeln und Datum hinzufügen
current_date = datetime.now().strftime("%Y-%m-%d")
output_file = 'redirects.txt'

with open(output_file, 'w') as f:
    f.write(f"# Rewrite rules generated on {current_date}\n")
    for rule in rewrite_rules:
        if rule:  # Nur gültige Rewrite-Regeln hinzufügen
            f.write(f"{rule}\n")

print("Die Datei 'redirects.txt' mit Rewrite-Regeln wurde erstellt.")

# Schritt 4: URLs für manuelle Überprüfung speichern
# Prüfen, ob auch bei diesen URLs 'origin' und 'destination' als relative URL identisch sind
df_manual_matching = df[
    ~((df['origin_language'] == df['matched_language']) &
    (df['similarity_score'] >= score_threshold))
]

# Filter für URLs, bei denen 'origin_url' und 'matched_url' als relative URL gleich sind
df_manual_matching['relative_origin'] = df_manual_matching['origin_url'].apply(convert_to_relative)
df_manual_matching['relative_matched'] = df_manual_matching['matched_url'].apply(convert_to_relative)

# URLs herausfiltern, bei denen 'relative_origin' nicht gleich 'relative_matched' ist
df_manual_matching_filtered = df_manual_matching[
    df_manual_matching['relative_origin'] != df_manual_matching['relative_matched']
]

# Datei url_for_manual_matching.csv speichern
df_manual_matching_filtered.to_csv('url_for_manual_matching.csv', index=False)

print("Die Datei 'url_for_manual_matching.csv' wurde aktualisiert.")

# Schritt 5: Dateien herunterladen
files.download('redirects.txt')
files.download('url_for_manual_matching.csv')


Saving url_for_rewrite_rules.csv to url_for_rewrite_rules (4).csv
                                          origin_url  \
0      https://www.neumeisterhydraulik.de/en/opt-out   
1  https://www.neumeisterhydraulik.de/en/home/pro...   
2  https://www.neumeisterhydraulik.de/en/home/app...   
3  https://www.neumeisterhydraulik.de/en/home/pro...   
4  https://www.neumeisterhydraulik.de/en/home/com...   

                                         matched_url similarity_score  \
0  https://neumeisterhydraulik.hotbytes.rocks/en/...       0.89248943   
1  https://neumeisterhydraulik.hotbytes.rocks/en/...        0.9632619   
2  https://neumeisterhydraulik.hotbytes.rocks/en/...        0.9440943   
3  https://neumeisterhydraulik.hotbytes.rocks/en/...        0.9682853   
4  https://neumeisterhydraulik.hotbytes.rocks/en/...        0.9128579   

  origin_language matched_language  Mapping korrekt  
0         English          English             True  
1         English          English             Tru

<ipython-input-5-e0dc759f4e48>:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_manual_matching['relative_origin'] = df_manual_matching['origin_url'].apply(convert_to_relative)
<ipython-input-5-e0dc759f4e48>:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_manual_matching['relative_matched'] = df_manual_matching['matched_url'].apply(convert_to_relative)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>